#### Exercise 1: Download the dataset

https://www.kaggle.com/kavita5/twitter-dataset-avengersendgame/download

In [1]:
import pandas as pd
import numpy as np
from pandas import Series,DataFrame
from sklearn import *

#### Exercise 2: 
Create Target column from retweetCount > np.median[retweetCount] 

Create dataset in .csv file with new features along with .yaml file with it's descritpion, median[retweetCount] value stored in retweetCount_median variable, preferably store data in data folder

In [2]:
# Add column to dataframe
tweets_df = pd.read_csv('./data/tweets.csv', encoding='cp1252')
tweets_df['High_Retweet_Count'] = tweets_df['retweetCount'] > np.median(tweets_df['retweetCount'])
tweets_df['High_Retweet_Count'].replace({False: 0, True: 1}, inplace=True)

%store tweets_df
tweets_df.head(1)

Stored 'tweets_df' (DataFrame)


,Unnamed: 0,text,favorited,favoriteCount,replyToSN,created,truncated,replyToSID,id,replyToUID,statusSource,screenName,retweetCount,isRetweet,retweeted,longitude,latitude,High_Retweet_Count
0,1,RT @mrvelstan: literally nobody:\r\nme:\r\n\r\...,False,0,NaN,2019-04-23 10:43:30,False,NaN,1120639328034676737,NaN,"<a href=""http://twitter.com/download/android"" ...",DavidAc96,637,True,False,NaN,NaN,0


In [3]:
# Save new csv
!mkdir data/new_tweets
tweets_df.to_csv('./data/new_tweets/new_tweets.csv')

mkdir: data/new_tweets: File exists


In [4]:
%%writefile './data/new_tweets/new_tweets.yaml'

# new_tweets.csv

description: This dataset is the same as ./data/tweets.csv with the addition of High_Retweet_Count column.
    Also median[retweetCount] value is stored in retweetCount_median variable.
    
retweetCount_median: 1755

Overwriting ./data/new_tweets/new_tweets.yaml


#### Exercise 3: Collaborate at github account

https://github.com/maxmmsu/harbour.space.DS.402

Create an .md file in it with the table of participants (one for a group), select a short acronim for your name (e.g. Maxim Musin --> mm) and write down the acronim to the table.

In [ ]:
# Created table row with my name

#### Exercise 4: Сreate a folder with your first exeperimet, name it exp_[your name acronim]_1, make it a python package, organize all transformers and classifiers wih python modules and .yaml files. 

In [5]:
# I create them later

#### Exercise 5: Select one of the following feature creation technics, write it down to collaboration table

##### Step 1 create 3 of the following features
- Extract lenght of the sentence
- Extract number of words
- Extract number of characters
- Extract number of hastags

- Create a dummy variable for RT at the beginning of the twit
- Creatu dummy variable from first letter of twit being A, B, C or D

##### Step 2 preform polynomial feature generation with parameter k = 2..6 

##### Step 3 perform dimensionality reduction to limit number of vatiables
- perform PCA dimensionality reduction
- *prefromr PCA dimensionality reduction according to covered variance
- select features with maximum mutual information with target variable
- *select features according to sum of adjusted mutual info score

### Step 1: 
#### Extract number of words

In [6]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import re

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/macbookpro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
def number_of_words(sentence):
    sentence_array = word_tokenize(sentence)
    words = [word for word in sentence_array if word.isalpha()]
    return len(words)

tweets_df['number_of_words'] = tweets_df['text'].apply(number_of_words)

#### Extract number of characters

In [8]:
def number_of_chars(sentence):
    return len(sentence)

tweets_df['number_of_chars'] = tweets_df['text'].apply(number_of_chars)

#### Extract number of hashtags

In [9]:
def number_of_hashtags(sentence):
    return len(re.findall('#\w+', sentence))

tweets_df['number_of_hashtags'] = tweets_df['text'].apply(number_of_hashtags)

In [11]:
tweets_df.head(1)
%store tweets_df

Stored 'tweets_df' (DataFrame)


### Step 2:
#### Polynomial features, degree = 3

In [12]:
# Check numerical features
list(tweets_df.select_dtypes(include=[np.number]).columns.values)

['Unnamed: 0',
 'favoriteCount',
 'replyToSID',
 'id',
 'replyToUID',
 'retweetCount',
 'longitude',
 'latitude',
 'High_Retweet_Count',
 'number_of_words',
 'number_of_chars',
 'number_of_hashtags']

In [13]:
# Check how many are null
tweets_df.isnull().sum(), tweets_df.shape

(Unnamed: 0                0
 text                      0
 favorited                 0
 favoriteCount             0
 replyToSN             14603
 created                   0
 truncated                 0
 replyToSID            14631
 id                        0
 replyToUID            14603
 statusSource              0
 screenName                0
 retweetCount              0
 isRetweet                 0
 retweeted                 0
 longitude             14996
 latitude              14996
 High_Retweet_Count        0
 number_of_words           0
 number_of_chars           0
 number_of_hashtags        0
 dtype: int64,
 (15000, 21))

In [14]:
numerical_features = [
 'favoriteCount',
#  'retweetCount', not used because is heavly corelated with the target, High_Retweet_Count
 'number_of_words',
 'number_of_chars',
 'number_of_hashtags']

poly = preprocessing.PolynomialFeatures(degree=3)

transformed_df = poly.fit_transform(tweets_df[numerical_features])

transformed_df.shape, type(transformed_df)

((15000, 35), numpy.ndarray)

### Step 3:
#### Perform PCA reduction

In [15]:
scaler = preprocessing.StandardScaler()
transformed_df = scaler.fit_transform(transformed_df)

pca = decomposition.PCA()
transformed_df = pca.fit_transform(transformed_df)

#### PCA reduction based on variance

In [16]:
# we need to keep components that have at least .9 variance
ratios_sum = 0
ratios_index = 0
ratios = list(pca.explained_variance_ratio_)

while np.sum(ratios[:ratios_index]) < 0.9 and ratios_index < len(ratios):
    ratios_index += 1
        
# So we choose only the first 4 features
pca = decomposition.PCA(n_components=ratios_index)
# transformed_df = pca.fit_transform(transformed_df)

transformed_df = pd.DataFrame(pca.fit_transform(transformed_df), columns=['PCA%i' % i for i in range(4)])

transformed_df

,PCA0,PCA1,PCA2,PCA3
0,-0.559881,-3.757546,0.974194,-0.020212
1,-0.218621,-0.998774,1.606653,0.036647
2,-0.533296,-3.506074,0.830690,-0.022568
3,-0.515959,-3.335006,0.714966,-0.027196
4,-0.306377,-1.773567,1.830159,0.029261
...,...,...,...,...
14995,-0.374711,-1.680291,-2.169746,-0.038321
14996,0.267378,4.330253,-4.685741,-0.167869
14997,0.185075,2.669166,0.194137,0.034035
14998,-0.306377,-1.773567,1.830159,0.029261


#### Select features with maximum mutual information with target variable

In [17]:
scores = feature_selection.mutual_info_classif(transformed_df, tweets_df['High_Retweet_Count'])
feature_scores = {feature: scores[feature] for feature in range(transformed_df.shape[1])}
feature_scores = ['PCA' + str(k) for k, v in sorted(feature_scores.items(), key=lambda item: item[1], reverse=True)]
best_features = feature_scores[0 : int(len(feature_scores) * .75)]

transformed_df[best_features]

,PCA2,PCA3,PCA0
0,0.974194,-0.020212,-0.559881
1,1.606653,0.036647,-0.218621
2,0.830690,-0.022568,-0.533296
3,0.714966,-0.027196,-0.515959
4,1.830159,0.029261,-0.306377
...,...,...,...
14995,-2.169746,-0.038321,-0.374711
14996,-4.685741,-0.167869,0.267378
14997,0.194137,0.034035,0.185075
14998,1.830159,0.029261,-0.306377


# EXPERIMENT TIME 🕵🏻‍♂️
##### Experiment 5_1

This experiment takes numerical features, generates new features using PolynomialFeatures, reduces features with PCA and classifies the data using Logistic Regression.

In [18]:
!mkdir experiments
!mkdir experiments/exp_5_1
!mkdir experiments/exp_5_1/result

mkdir: experiments: File exists
mkdir: experiments/exp_5_1: File exists
mkdir: experiments/exp_5_1/result: File exists


In [19]:
%%writefile experiments/exp_5_1/config.yaml

description: This experiment takes numerical features, generates new features using PolynomialFeatures, reduces features with PCA and classifies the data using Logistic Regression.
    
features: favoriteCount, number_of_words, number_of_chars, number_of_hashtags
target: High_Retweet_Count
    
config_variables:
    - logistic_penalty
    - logistic_rand_state
    - target
    - pca_components
    - poly_degree
    
logistic_penalty: l2
logistic_rand_state: 2020
pca_components: None
poly_degree: 3

Overwriting experiments/exp_5_1/config.yaml


In [20]:
%%writefile experiments/exp_5_1/transformer.py
import numpy as np 
import pandas as pd
import nltk
import unicodedata
import re
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from nltk.tokenize import word_tokenize

from experiments.base.transformer import BaseTransformer


class Transformer_5_1(BaseTransformer):
    def __init__(self):
        nltk.download('punkt')
        return None
    
    def number_of_words(self, sentence):
        sentence_array = word_tokenize(sentence)
        words = [word for word in sentence_array if word.isalpha()]
        return len(words)
    
    def number_of_chars(self, sentence):
        return len(sentence)

    def number_of_hashtags(self, sentence):
        return len(re.findall('#\w+', sentence))
    
    def fit_transform(self, X):
        X['number_of_words'] = X['text'].apply(self.number_of_words)
        X['number_of_chars'] = X['text'].apply(self.number_of_chars)
        X['number_of_hashtags'] = X['text'].apply(self.number_of_hashtags)

        poly = preprocessing.PolynomialFeatures(degree=3)
        transformed_df = poly.fit_transform(X.drop('text', axis=1))
        return transformed_df

Overwriting experiments/exp_5_1/transformer.py


In [23]:
%%writefile experiments/exp_5_1/main.py
import pandas as pd
import numpy as np
import pickle
from experiments.exp_5_1.transformer import Transformer_5_1
from experiments.base.classifier import LogisticCustomClassifier

X_train, X_test, y_train, y_test = pickle.load(open('./data/split/split.pickle', 'rb'))

features = ['text', 'favoriteCount']
target = 'High_Retweet_Count'

classifier = LogisticCustomClassifier('experiments/exp_5_1/config.yaml', Transformer_5_1, 'experiments/exp_5_1/result/')
classifier.fit(X_train[features], y_train[target])
y_pred = classifier.predict(X_test[features])
classifier.generate_results(y_pred, y_test[target])

Overwriting experiments/exp_5_1/main.py


In [24]:
%%time
%run experiments/exp_5_1/main.py

Using configuration file experiments/exp_5_1/config.yaml:


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/macbookpro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/opt/anaconda3/envs/python37/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/macbookpro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


------------------
{'pca_components': 34, 'accuracy': 0.6666666666666666, 'precision': 0.7267796610169491, 'recall': 0.6423007789095266, 'mutual_info_score': 0.08410396469187705}
------------------
Results are saved to: experiments/exp_5_1/result/result_02-Nov-2020-19-58-05.yaml
CPU times: user 4.34 s, sys: 27.3 ms, total: 4.37 s
Wall time: 3.39 s


#### Exercise 6: Extract names of avengers endgame characters from text 

Examples of tags in quetion:
#Thanos 
#WinterSoldier 
#CaptainAmerica 
#blackwidow
#CaptainMarvel
#Mantis 

store the resulting dataset in data folder with .yaml description, preferable naming for variable names is lowercase name of the tag

In [4]:
%store -r tweets_df

import re

all_text = " ".join(list(tweets_df['text']))

tags_dict = {}
heroes = ["Iron Man", "Thor", "Captain America", "Captain Marvel", "Black Widow", "Hawkeye", "Hulk", "Vision", "Scarlet Witch", "War Machine", "Falcon", "StarLord", "Rocket Raccoon", "Groot", "Gamora", "Drax", "Mantis", "Nebula", "Doctor Strange", "Wong", "Spider-Man", "Spiderman", "Winter Soldier", "Heimdall", "Black Panther", "Okoye", "Shuri", "M’Baku", "Eitiri", "Nick Fury", "Maria Hill", "Pepper Potts", "William “Thunderbolt” Ross", "Ned", "Thanos", "Loki", "the Collector", "Cull Obsidian", "Ebony Maw", "Proxima Midnight", "Corvus Glaive", "Red Skull", "The Wasp"]
heroes = [h.lower().replace(" ", "") for h in heroes]

print("Number of heroes: " + str(len(heroes)))

# gather all hashtags
for tag in re.findall('#\w+', all_text):
    tag = tag[1:].lower().replace(" ", "")
    if tag in tags_dict:
        tags_dict[tag] += 1
    else:
        tags_dict[tag] = 1
        
# remove the non-heroes hashtags
for tag in list(tags_dict):
    if tag not in heroes:
        del tags_dict[tag]
        
# change the dict for usage in dataframe
index = 0
for tag in list(tags_dict):
    index += 1
    tags_dict[index] = [tag, tags_dict.pop(tag)]
        
# create dataframe from dictionary of heroes
tags_df = pd.DataFrame.from_dict(tags_dict, orient='index', columns=['Hashtag', 'Count'])

tags_df.head()

Number of heroes: 43


,Hashtag,Count
1,blackwidow,567
2,captainamerica,1019
3,wintersoldier,160
4,thanos,176
5,thor,159


In [5]:
!mkdir ./data/hashtags

mkdir: ./data/hashtags: File exists


In [6]:
import pickle
pickle.dump(tags_df, open('./data/hashtags/hashtags.pickle', 'wb'))

In [7]:
%%writefile './data/hashtags/hashtags.yaml'

description: This dataset consists of hashtags that contain the names of the characters in Avengers Endgame movie
    
columns: 
    - Hashtag
    - Count

Overwriting ./data/hashtags/hashtags.yaml


#### Exercise 7: Create set of classifiers for logistic regression with different features for feature generation. 
use the following methods of feature generation

 - word tokenization with 1-gram, f
 - pos tokenization with extraction of all NP 
 - pos tokenization with 2-gram bag of tokens for all NNP
 - 2-gram bag of characters for screenName features
 - polynomial features for presence of avenger character names 
 
Organize classes of transformers in various .py files

In [10]:
# init txt_df
tweets_txt_df = tweets_df[['text']]
tweets_txt_df.head()

%store tweets_txt_df

Stored 'tweets_txt_df' (DataFrame)


#### Clean the text

In [43]:
import nltk
import unicodedata
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
pd.options.mode.chained_assignment = None

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/macbookpro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/macbookpro/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/macbookpro/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [40]:
def clean_text(text):
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words('english') + ['RT']
    text = (unicodedata.normalize('NFKD', text)
        .encode('ascii', 'ignore')
        .decode('utf-8', 'ignore'))
    words = re.sub(r'[^\w\s]', '', text) 
    words = re.sub(r'(http.+)', '', words).split()
    return ' '.join([word for word in words if word not in stopwords])
#     return ' '.join([wnl.lemmatize(word) for word in words if word not in stopwords])

In [12]:
tweets_txt_df['text'] = tweets_txt_df['text'].apply(clean_text)
tweets_txt_df.head(2)

,text
0,mrvelstan literally nobody AvengersEndgame
1,agntecarter im emotional sorry 2014 x 2019 bla...


### 1-gram tokenization

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorized_txt_df = vectorizer.fit_transform(tweets_txt_df['text'])

vectorized_txt_df.shape, tweets_txt_df.shape, vectorizer.get_feature_names()[300:302]

((15000, 5888), (15000, 1), ['aftershocks', 'afterwards'])

# EXPERIMENT TIME👨🏻‍🔬
##### Experiment 7_1

This experiment uses 1-gram tokenization and logistic regression for classification. 

In [20]:
!mkdir experiments
!mkdir experiments/exp_7_1
!mkdir experiments/exp_7_1/result

mkdir: experiments: File exists
mkdir: experiments/exp_7_1: File exists
mkdir: experiments/exp_7_1/result: File exists


In [21]:
%%writefile experiments/exp_7_1/__init__.py

# just an empty python file

Overwriting experiments/exp_7_1/__init__.py


In [22]:
%%writefile experiments/exp_7_1/config.yaml

description: This experiment uses 1-gram tokenization and logistic regression for classification. 
    Choosing [None] for pca_components will automate the process by selecting the components with sum variance greater than .9
    
features: 1-gram tokenization of text
target: High_Retweet_Count
    
config_variables:
    - logistic_penalty
    - logistic_rand_state
    - target
    - pca_components
    
logistic_penalty: l2
logistic_rand_state: 2020
pca_components: None

Overwriting experiments/exp_7_1/config.yaml


In [23]:
%%writefile experiments/exp_7_1/transformer.py
### Custom Transformer for 1-gram tokenization
import numpy as np 
import pandas as pd
import nltk
import unicodedata
import re
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer

from experiments.base.transformer import BaseTransformer


class Transformer_7_1(BaseTransformer):
    def __init__(self):
        return None

    def fit_transform(self, X):
        X = X.apply(self.clean_text)
        vectorizer = CountVectorizer()
        transformer_df = vectorizer.fit_transform(X)
        return transformer_df

Overwriting experiments/exp_7_1/transformer.py


In [24]:
%%writefile experiments/exp_7_1/main.py
import pandas as pd
import numpy as np
import pickle
from experiments.exp_7_1.transformer import Transformer_7_1
from experiments.base.classifier import LogisticCustomClassifier

X_train, X_test, y_train, y_test = pickle.load(open('./data/split/split.pickle', 'rb'))

features = 'text'
target = 'High_Retweet_Count'

classifier = LogisticCustomClassifier('experiments/exp_7_1/config.yaml', Transformer_7_1, 'experiments/exp_7_1/result/')
classifier.fit(X_train[features], y_train[target])
y_pred = classifier.predict(X_test[features])
classifier.generate_results(y_pred, y_test[target])

Overwriting experiments/exp_7_1/main.py


In [29]:
%%time
%run experiments/exp_7_1/main.py

Using configuration file experiments/exp_7_1/config.yaml:
------------------
{'pca_components': 633, 'accuracy': 0.5606666666666666, 'precision': 0.5030508474576271, 'recall': 0.5591559909570459, 'mutual_info_score': 0.01026291882307718}
------------------
Results are saved to: experiments/exp_7_1/result/result_02-Nov-2020-20-35-14.yaml
CPU times: user 6min 18s, sys: 4.56 s, total: 6min 23s
Wall time: 1min 40s


### Pos tokenization with extraction of all NN

In [30]:
tweets_txt_df.head()

,text
0,mrvelstan literally nobody AvengersEndgame
1,agntecarter im emotional sorry 2014 x 2019 bla...
2,saving bingo cards tomorrow AvengersEndgame
3,HelloBoon Man AvengersEndgame ads everywhere
4,Marvel We salute ChrisEvans CaptainAmerica Ave...


In [31]:
nltk.download('tagsets')
nltk.help.upenn_tagset('NN')
nltk.help.upenn_tagset('NNP')

NN: noun, common, singular or mass
    common-carrier cabbage knuckle-duster Casino afghan shed thermostat
    investment slide humour falloff slick wind hyena override subhumanity
    machinist ...
NNP: noun, proper, singular
    Motown Venneboerger Czestochwa Ranzer Conchita Trumplane Christos
    Oceanside Escobar Kreisler Sawyer Cougar Yvette Ervin ODI Darryl CTCA
    Shannon A.K.C. Meltex Liverpool ...


[nltk_data] Downloading package tagsets to
[nltk_data]     /Users/macbookpro/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


In [32]:
from nltk import pos_tag
from nltk import word_tokenize

# tokenize words
tweets_txt_df['tokens'] = tweets_txt_df['text'].apply(word_tokenize)

# pos tokenize words
tweets_txt_df['pos_tokens'] = tweets_txt_df['tokens'].apply(pos_tag)

# extract nn
tweets_txt_df['nn_tokens'] = tweets_txt_df['pos_tokens'].apply(lambda token_list: [token[0] for token in token_list if token[1] == 'NN'])

# make nn into single strings
tweets_txt_df['nn_tokens'] = tweets_txt_df['nn_tokens'].apply(lambda x: ' '.join(x))

# show nn strings
tweets_txt_df['nn_tokens'].head(3)

0                                     
1    sorry x blackwidow captainamerica
2                       bingo tomorrow
Name: nn_tokens, dtype: object

In [33]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorized_nn_df = vectorizer.fit_transform(tweets_txt_df['nn_tokens'])

vectorized_nn_df.shape, tweets_txt_df.shape

((15000, 1700), (15000, 4))

# EXPERIMENT TIME👨🏻‍🔬
##### Experiment 7_2

This experiment uses pos tokenization with extraction of NN and logistic regression for classification.

In [34]:
!mkdir experiments
!mkdir experiments/exp_7_2
!mkdir experiments/exp_7_2/result

mkdir: experiments: File exists
mkdir: experiments/exp_7_2: File exists
mkdir: experiments/exp_7_2/result: File exists


In [35]:
%%writefile experiments/exp_7_2/__init__.py

# just an empty python file

Overwriting experiments/exp_7_2/__init__.py


In [6]:
%%writefile experiments/exp_7_2/config.yaml

description: This experiment uses pos tokenization with extraction of NN and logistic regression for classification. 
    Choosing None for pca_components will automate the process by selecting the components with sum variance greater than .9
    
features: pos tokenization with extraction of NN
target: High_Retweet_Count
    
config_variables:
    - logistic_penalty
    - logistic_rand_state
    - pca_components
    - target
    
logistic_penalty: l2
logistic_rand_state: 2020
pca_components: None

Overwriting experiments/exp_7_2/config.yaml


In [3]:
%%writefile experiments/exp_7_2/transformer.py
import numpy as np 
import pandas as pd
import nltk
import unicodedata
import re
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from nltk import pos_tag
from nltk import word_tokenize

from experiments.base.transformer import BaseTransformer


class Transformer_7_2(BaseTransformer):
    def __init__(self):
        nltk.download('tagsets')
        return None

    def fit_transform(self, X):        
        X['text'] = X['text'].apply(self.clean_text)
        
        # tokenize words
        X['tokens'] = X['text'].apply(word_tokenize)

        # pos tokenize words
        X['pos_tokens'] = X['tokens'].apply(pos_tag)

        # extract nn
        X['nn_tokens'] = X['pos_tokens'].apply(lambda token_list: [token[0] for token in token_list if token[1] == 'NN'])

        # make nn into single strings
        X['nn_tokens'] = X['nn_tokens'].apply(lambda x: ' '.join(x))
        
        vectorizer = CountVectorizer()
        transformer_df = vectorizer.fit_transform(X['nn_tokens'])
        return transformer_df

Overwriting experiments/exp_7_2/transformer.py


In [7]:
%%writefile experiments/exp_7_2/main.py
import pandas as pd
import numpy as np
import pickle
from experiments.exp_7_2.transformer import Transformer_7_2
from experiments.base.classifier import LogisticCustomClassifier

X_train, X_test, y_train, y_test = pickle.load(open('./data/split/split.pickle', 'rb'))

features = 'text'
target = 'High_Retweet_Count'

classifier = LogisticCustomClassifier('experiments/exp_7_2/config.yaml', Transformer_7_2, 'experiments/exp_7_2/result/')
classifier.fit(X_train, y_train[target])
y_pred = classifier.predict(X_test)
classifier.generate_results(y_pred, y_test[target])

Overwriting experiments/exp_7_2/main.py


In [8]:
%%time
%run experiments/exp_7_2/main.py

Using configuration file experiments/exp_7_2/config.yaml:


[nltk_data] Downloading package tagsets to
[nltk_data]     /Users/macbookpro/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     /Users/macbookpro/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


------------------
{'pca_components': 523, 'accuracy': 0.6406666666666667, 'precision': 0.8074576271186441, 'recall': 0.6, 'mutual_info_score': 0.07050774596867938}
------------------
Results are saved to: experiments/exp_7_2/result/result_02-Nov-2020-21-05-18.yaml
CPU times: user 34 s, sys: 1.1 s, total: 35.1 s
Wall time: 19.5 s


### Pos tokenization with 2-gram bag of tokens for all NNP

In [40]:
from nltk import pos_tag
from nltk import word_tokenize

# tokenize words
tweets_txt_df['tokens'] = tweets_txt_df['text'].apply(word_tokenize)

# pos tokenize words
tweets_txt_df['pos_tokens'] = tweets_txt_df['tokens'].apply(pos_tag)

# extract nnp
tweets_txt_df['nnp_tokens'] = tweets_txt_df['pos_tokens'].apply(lambda token_list: [token[0] for token in token_list if token[1] == 'NNP'])

# make nnnp into single strings
tweets_txt_df['nnp_tokens'] = tweets_txt_df['nnp_tokens'].apply(lambda x: ' '.join(x))

# show nnbp strings
tweets_txt_df['nnp_tokens'].head()

0                          AvengersEndgame
1                                         
2                                         
3            HelloBoon Man AvengersEndgame
4    Marvel CaptainAmerica AvengersEndgame
Name: nnp_tokens, dtype: object

In [42]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(2, 2))
vectorized_nnp_df = vectorizer.fit_transform(tweets_txt_df['nnp_tokens'])

vectorized_nnp_df.shape, vectorizer.get_feature_names()[100:102]

((15000, 5471), ['alienware m15', 'alilchubbypod thursday'])

# EXPERIMENT TIME👨🏻‍🔬
##### Experiment 7_3

This experiment uses pos tokenization with extraction of NNP (2-gram) and logistic regression for classification.

In [9]:
!mkdir experiments
!mkdir experiments/exp_7_3
!mkdir experiments/exp_7_3/result

mkdir: experiments: File exists
mkdir: experiments/exp_7_3: File exists
mkdir: experiments/exp_7_3/result: File exists


In [14]:
%%writefile experiments/exp_7_3/config.yaml

description: This experiment uses pos tokenization with extraction of NNP (2-gram) and logistic regression for classification.
    Choosing [None] for pca_components will automate the process by selecting the components with sum variance > .9
    
features: pos tokenization with extraction of NN
target: High_Retweet_Count
    
config_variables:
    - logistic_penalty
    - logistic_rand_state
    - pca_components
    - target
    
logistic_penalty: l1
logistic_rand_state: 2020
pca_components: None

Overwriting experiments/exp_7_3/config.yaml


In [15]:
%%writefile experiments/exp_7_3/main.py
import pandas as pd
import numpy as np
import pickle
from experiments.exp_7_3.transformer import Transformer_7_3
from experiments.base.classifier import LogisticCustomClassifier

X_train, X_test, y_train, y_test = pickle.load(open('./data/split/split.pickle', 'rb'))

features = 'text'
target = 'High_Retweet_Count'

classifier = LogisticCustomClassifier('experiments/exp_7_3/config.yaml', Transformer_7_3, 'experiments/exp_7_3/result/')
classifier.fit(X_train, y_train[target])
y_pred = classifier.predict(X_test)
classifier.generate_results(y_pred, y_test[target])

Overwriting experiments/exp_7_3/main.py


In [16]:
%%time
%run experiments/exp_7_3/main.py

Using configuration file experiments/exp_7_3/config.yaml:


[nltk_data] Downloading package tagsets to
[nltk_data]     /Users/macbookpro/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
/opt/anaconda3/envs/python37/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[nltk_data] Downloading package tagsets to
[nltk_data]     /Users/macbookpro/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


------------------
{'pca_components': 721, 'accuracy': 0.6903333333333334, 'precision': 0.7294915254237289, 'recall': 0.6699875466998755, 'mutual_info_score': 0.1086167869256012}
------------------
Results are saved to: experiments/exp_7_3/result/result_02-Nov-2020-21-27-57.yaml
CPU times: user 1min 24s, sys: 1.78 s, total: 1min 26s
Wall time: 40 s


### 2-gram bag of characters for screenName features

In [18]:
%store -r tweets_df

In [19]:
tweets_df['screenName']

0              DavidAc96
1                NRmalaa
2               jijitsuu
3               SahapunB
4            stella22_97
              ...       
14995          tommysboi
14996    kimberleywithae
14997         Gnanavel07
14998         _moonljght
14999    CaterinaCabrel1
Name: screenName, Length: 15000, dtype: object

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 2))
vectorized_screenName_df = vectorizer.fit_transform(tweets_df['screenName'])

vectorized_screenName_df.shape, vectorizer.get_feature_names(), 

((15000, 1271),
 ['00',
  '01',
  '02',
  '03',
  '04',
  '05',
  '06',
  '07',
  '08',
  '09',
  '0_',
  '0a',
  '0b',
  '0c',
  '0d',
  '0f',
  '0h',
  '0i',
  '0k',
  '0l',
  '0m',
  '0n',
  '0o',
  '0p',
  '0r',
  '0s',
  '0t',
  '0u',
  '0v',
  '0w',
  '0x',
  '0y',
  '10',
  '11',
  '12',
  '13',
  '14',
  '15',
  '16',
  '17',
  '18',
  '19',
  '1_',
  '1a',
  '1b',
  '1c',
  '1d',
  '1e',
  '1f',
  '1h',
  '1j',
  '1k',
  '1l',
  '1m',
  '1n',
  '1p',
  '1q',
  '1r',
  '1s',
  '1t',
  '1v',
  '20',
  '21',
  '22',
  '23',
  '24',
  '25',
  '26',
  '27',
  '28',
  '29',
  '2_',
  '2a',
  '2b',
  '2c',
  '2d',
  '2e',
  '2f',
  '2g',
  '2i',
  '2j',
  '2k',
  '2m',
  '2n',
  '2p',
  '2q',
  '2r',
  '2s',
  '2u',
  '2v',
  '2w',
  '2y',
  '2z',
  '30',
  '31',
  '32',
  '33',
  '34',
  '35',
  '36',
  '37',
  '38',
  '39',
  '3_',
  '3a',
  '3b',
  '3c',
  '3d',
  '3e',
  '3f',
  '3h',
  '3k',
  '3l',
  '3n',
  '3p',
  '3q',
  '3r',
  '3s',
  '3t',
  '3u',
  '3v',
  '3w',
  '3y',


# EXPERIMENT TIME👨🏻‍🔬
##### Experiment 7_4

This experiment uses 2-gram bag of chars for screenName feature and logistic regression for classification.

In [21]:
!mkdir experiments
!mkdir experiments/exp_7_4
!mkdir experiments/exp_7_4/result

mkdir: experiments: File exists
mkdir: experiments/exp_7_4: File exists
mkdir: experiments/exp_7_4/result: File exists


In [26]:
%%writefile experiments/exp_7_4/config.yaml

description: This experiment uses 2-gram bag of chars for screenName feature and logistic regression for classification.
    Choosing [None] for pca_components will automate the process by selecting the components with sum variance > .9
    
features: characters paired 2 by 2
target: High_Retweet_Count
    
config_variables:
    - logistic_penalty
    - logistic_rand_state
    - pca_components
    - target
    
logistic_penalty: l1
logistic_rand_state: 2020
pca_components: None

Overwriting experiments/exp_7_4/config.yaml


In [23]:
%%writefile experiments/exp_7_4/transformer.py
import numpy as np 
import pandas as pd
import nltk
import unicodedata
import re
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from nltk import pos_tag
from nltk import word_tokenize

from experiments.base.transformer import BaseTransformer


class Transformer_7_4(BaseTransformer):
    def __init__(self):
        nltk.download('tagsets')
        return None

    def fit_transform(self, X):
        vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 2))
        transformer_df = vectorizer.fit_transform(X['screenName'])
        return transformer_df

Overwriting experiments/exp_7_4/transformer.py


In [24]:
%%writefile experiments/exp_7_4/main.py
import pandas as pd
import numpy as np
import pickle
from experiments.exp_7_4.transformer import Transformer_7_4
from experiments.base.classifier import LogisticCustomClassifier

X_train, X_test, y_train, y_test = pickle.load(open('./data/split/split.pickle', 'rb'))

features = 'screenName'
target = 'High_Retweet_Count'

classifier = LogisticCustomClassifier('experiments/exp_7_4/config.yaml', Transformer_7_4, 'experiments/exp_7_4/result/')
classifier.fit(X_train, y_train[target])
y_pred = classifier.predict(X_test)
classifier.generate_results(y_pred, y_test[target])

Overwriting experiments/exp_7_4/main.py


In [27]:
%%time
%run experiments/exp_7_4/main.py

Using configuration file experiments/exp_7_4/config.yaml:


[nltk_data] Downloading package tagsets to
[nltk_data]     /Users/macbookpro/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     /Users/macbookpro/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


------------------
{'pca_components': 454, 'accuracy': 0.5016666666666667, 'precision': 0.5098305084745762, 'recall': 0.49343832020997375, 'mutual_info_score': -0.00023131604942293197}
------------------
Results are saved to: experiments/exp_7_4/result/result_02-Nov-2020-21-30-01.yaml
CPU times: user 21.7 s, sys: 520 ms, total: 22.3 s
Wall time: 7.94 s


### polynomial features for presence of avenger character names

In [28]:
from sklearn.feature_extraction.text import CountVectorizer

%store -r tweets_df
tweets_txt_df = tweets_df[['text']]

heroes = ["Iron Man", "Thor", "Captain America", "Captain Marvel", "Black Widow", "Hawkeye", "Hulk", "Vision", "Scarlet Witch", "War Machine", "Falcon", "StarLord", "Rocket Raccoon", "Groot", "Gamora", "Drax", "Mantis", "Nebula", "Doctor Strange", "Wong", "Spider-Man", "Spiderman", "Winter Soldier", "Heimdall", "Black Panther", "Okoye", "Shuri", "M’Baku", "Eitiri", "Nick Fury", "Maria Hill", "Pepper Potts", "William “Thunderbolt” Ross", "Ned", "Thanos", "Loki", "the Collector", "Cull Obsidian", "Ebony Maw", "Proxima Midnight", "Corvus Glaive", "Red Skull", "The Wasp"]
heroes = [h.lower().replace(" ", "") for h in heroes]

cv = CountVectorizer(vocabulary=heroes)

heroes_count_df = pd.DataFrame.sparse.from_spmatrix(cv.fit_transform(tweets_txt_df['text']), 
                   tweets_txt_df.index,
                   cv.get_feature_names())

In [29]:
heroes_count_arr = heroes_count_df.to_numpy()

In [30]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=3)
transformed_df = poly.fit_transform(heroes_count_arr)

transformed_df.shape

(15000, 15180)

# EXPERIMENT TIME👨🏻‍🔬
##### Experiment 7_5

This experiment uses polynomial features for presence of hero name in text feature and logistic regression for classification.

In [31]:
!mkdir experiments
!mkdir experiments/exp_7_5
!mkdir experiments/exp_7_5/result

mkdir: experiments: File exists
mkdir: experiments/exp_7_5: File exists
mkdir: experiments/exp_7_5/result: File exists


In [32]:
%%writefile experiments/exp_7_5/config.yaml

description: This experiment uses polynomial features for presence of hero name in text feature and logistic regression for classification.
    Choosing [None] for pca_components will automate the process by selecting the components with sum variance > .9
    
features: polynomial features of dummy variables
target: High_Retweet_Count
    
config_variables:
    - logistic_penalty
    - logistic_rand_state
    - pca_components
    - target
    
logistic_penalty: l2
logistic_rand_state: 2020
pca_components: 10

Overwriting experiments/exp_7_5/config.yaml


In [33]:
%%writefile experiments/exp_7_5/transformer.py
import numpy as np 
import pandas as pd
import nltk
import unicodedata
import re
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import PolynomialFeatures
from nltk import pos_tag
from nltk import word_tokenize

from experiments.base.transformer import BaseTransformer


class Transformer_7_5(BaseTransformer):
    def __init__(self):
        return None

    def fit_transform(self, X):
        X = X[['text']]

        heroes = ["Iron Man", "Thor", "Captain America", "Captain Marvel", "Black Widow", "Hawkeye", "Hulk", "Vision", "Scarlet Witch", "War Machine", "Falcon", "StarLord", "Rocket Raccoon", "Groot", "Gamora", "Drax", "Mantis", "Nebula", "Doctor Strange", "Wong", "Spider-Man", "Spiderman", "Winter Soldier", "Heimdall", "Black Panther", "Okoye", "Shuri", "M’Baku", "Eitiri", "Nick Fury", "Maria Hill", "Pepper Potts", "William “Thunderbolt” Ross", "Ned", "Thanos", "Loki", "the Collector", "Cull Obsidian", "Ebony Maw", "Proxima Midnight", "Corvus Glaive", "Red Skull", "The Wasp"]
        heroes = [h.lower().replace(" ", "") for h in heroes]

        cv = CountVectorizer(vocabulary=heroes)

        heroes_count_df = pd.DataFrame.sparse.from_spmatrix(cv.fit_transform(X['text']), 
                           X.index,
                           cv.get_feature_names())
        
        heroes_count_arr = heroes_count_df.to_numpy()
        
        poly = PolynomialFeatures(degree=3)
        transformed_df = poly.fit_transform(heroes_count_arr)
        return transformed_df

Overwriting experiments/exp_7_5/transformer.py


In [34]:
%%writefile experiments/exp_7_5/main.py
import pandas as pd
import numpy as np
import pickle
from experiments.exp_7_5.transformer import Transformer_7_5
from experiments.base.classifier import LogisticCustomClassifier

X_train, X_test, y_train, y_test = pickle.load(open('./data/split/split.pickle', 'rb'))

features = 'text'
target = 'High_Retweet_Count'

classifier = LogisticCustomClassifier('experiments/exp_7_5/config.yaml', Transformer_7_5, 'experiments/exp_7_5/result/')
classifier.fit(X_train, y_train[target])
y_pred = classifier.predict(X_test)
classifier.generate_results(y_pred, y_test[target])

Overwriting experiments/exp_7_5/main.py


In [35]:
%%time
%run experiments/exp_7_5/main.py

Using configuration file experiments/exp_7_5/config.yaml:


/opt/anaconda3/envs/python37/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


------------------
{'pca_components': 10, 'accuracy': 0.38533333333333336, 'precision': 0.7437288135593221, 'recall': 0.4280140460397971, 'mutual_info_score': 0.09368617357974225}
------------------
Results are saved to: experiments/exp_7_5/result/result_02-Nov-2020-21-49-42.yaml
CPU times: user 1h 7min 32s, sys: 38.5 s, total: 1h 8min 10s
Wall time: 17min 18s


#### Exercise 8: Apply PCA dimensionality reduction and LogisticRegression to predict Target, construct pipelines for all transformers from Exercise 5 and 7 (name them exp_[your name acronim]\_6_1, ... exp_[your name acronim]\_6_...) implement them as custom classifiers.

In [ ]:
# I did experiements 5_1, 7_1, 7_2, 7_3, 7_4, 7_5

#### Exercise 9: Split dataset to train and test (it is up to you which features you will include in it) and store it data folder along with .yaml description

In [37]:
%store -r tweets_df
tweets_df.head(1)

,Unnamed: 0,text,favorited,favoriteCount,replyToSN,created,truncated,replyToSID,id,replyToUID,...,screenName,retweetCount,isRetweet,retweeted,longitude,latitude,High_Retweet_Count,number_of_words,number_of_chars,number_of_hashtags
0,1,RT @mrvelstan: literally nobody:\r\nme:\r\n\r\...,False,0,NaN,2019-04-23 10:43:30,False,NaN,1120639328034676737,NaN,...,DavidAc96,637,True,False,NaN,NaN,0,7,81,1


In [38]:
%mkdir ./data/split

mkdir: ./data/split: File exists


In [46]:
import re
import pickle
from sklearn.model_selection import train_test_split

features = ['text', 'favoriteCount', 'screenName']
target = ['High_Retweet_Count']

tweets_df['text'] = tweets_df['text'].apply(clean_text)

X_train, X_test, y_train, y_test = train_test_split(tweets_df[features], \
                                                    tweets_df[target], \
                                                    test_size=0.2, \
                                                    random_state=2020)

pickle.dump(train_test_split(tweets_df[features], \
                                tweets_df[target], \
                                test_size=0.2, \
                                random_state=2020), open('./data/split/split.pickle', 'wb'))

In [47]:
%%writefile ./data/split/config.yaml

Description: Split for ./data/new_tweets/new_tweets.csv
    
test_size: 0.2
random_state: 2020

Overwriting ./data/split/config.yaml


In [48]:
X_train, X_test, y_train, y_test = pickle.load(open('./data/split/split.pickle', 'rb'))

X_train

13698    BrandonDavisBD AvengersEndgame astonishing ama...
1142     Marvel Black Widow Scarlett Johansson BlackWid...
7954     caplovesfondue Chris Evans brought WHOLE FAMIL...
2225     targrycn honestly chris evans saying cried six...
9350     Marvel Black Widow Scarlett Johansson BlackWid...
                               ...                        
11971    Us trying book AvengersEndgame weekend hammad1998
14966         kookiewoman part journey end AvengersEndgame
7491     PNemiroff AvengersEndgame A LOT Too much times...
12680    Factory Farmed Salmon Full Disease Hazardous C...
9056     hmvtweets WIN AvengersEndgame merch bundle Sim...
Name: text, Length: 12000, dtype: object

#### Exercise 10: Add intialization from .yaml descriptions of classifiers to implementations of classifiers at Exercise 7. 

In [ ]:
# Done

#### Exercise 11: Train classifiers with various PCA dimensionality, bag of words and polynomial paramters paramteres on train, test them on test and store in .yaml files for every experiment with resulting metrics

- accuracy
- precision
- recall
- adjusted_mutual_information

In [ ]:
# Done

#### Exercise 12*: Read urls on dataset follow them with the following example, download attached images and count people on them with cv features

#### Exercise 13*: Add new features to all PCA output on Exercise 6, train classifiers, test quality and store results in proper .yaml files

#### Exercise 14*: Perform topic modelings on text feature, add it to PCA features from Exercise 8, train classifiers, test quality and store results in proper .yaml files

#### Exercise 15: Combine all combinations of features from Exercise 8, measure the results of improvement of classifier

#### Exercise 16*: For 5 best of prevous experiments, change LogisticRegression to XGBClassifier. Try several XGB configurations, store results and parameters in .yaml files

#### Exercise 17: in collaboration with classmates create the best possible classifier for the data using various technics